## Explore whether LLMs can identify waiting

I'm curious to know how well an LLM can identify things that a patient might be waiting for, from unstructured text in patient notes. Below are a couple of examples of prompts that I used to extract this information using (1) Universal NER and (2) LLama2.

UniversalNER exists to recognise entities, like treatments, but can't specific whether they have been provided or not. 

Clearly, a transformer-based model like llama2 benefits from more weights and more training data, so does much better. With llama2, I found that a more detailed prompt asking for actions pending worked better

In [2]:
notebook_name = 'explore_waiting'

In [69]:
import os
import pandas as pd
import sys
import json

from pathlib import Path

from langchain.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.llms import LlamaCpp

# Import the variables that have been set in the init.py folder in the root directory
# These include a constant called PROJECT_ROOT which stores the absolute path to this folder
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import init
PROJECT_ROOT = os.getenv("PROJECT_ROOT")

# Add the src folder to sys path, so that the application knows to look there for libraries
sys.path.append(str(Path(PROJECT_ROOT) / 'src'))

In [75]:
# for Universal NER
from langchain.llms import LlamaCpp
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Max length for NER
max_length = 512

### Load data from other experiments

In [6]:
# unstuctured data
with open('../src/data_exports/explore_llama_20240225_1202.json') as f:
    notes_data = json.load(f)

### Using UniversalNER to identify what patient is waiting for

In [101]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

n_gpu_layers = 1
n_batch = 412
llm_ner = LlamaCpp(
    model_path="../../llama.cpp/models/quantized_q4_1.gguf",
    n_gpu_layers=n_gpu_layers,
    n_batch=n_batch,
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    callback_manager=callback_manager,
    verbose=True
)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from ../../llama.cpp/models/quantized_q4_1.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_1     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_1     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q4_1     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q4_1     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q4_1     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q4_1     [  4096, 11008,     1,     1 ]
llama_model_loader: - tensor    6:            blk.0.ffn_down.weight q4_1     [ 11008,  4096,     1,     1 ]
llama_model_loader: - tensor    7:              blk.0.ffn_up.weight q4_1     [  4096, 11008,

In [74]:
# A function to shorten the note
def shorten_note(patient_note, instructions, max_length):

    # Tokenize the instructions to find out how long they are
    instructions_tokens = tokenizer(instructions, truncation=True, return_tensors="pt")

    # count tokens in patient note 
    patient_note_tokens = tokenizer(patient_note, truncation=True, return_tensors="pt")
    num_patient_note_tokens = len(patient_note_tokens['input_ids'][0])

    # work out how many tokens the patient note can occupy
    truncate_note_tokens_at = max_length - len(instructions_tokens['input_ids'][0]) 

    # find which word to truncate the non-tokenized input at, by (clumsily) taking the proportion of the two token lengths
    truncate_note_words_at = round(len(patient_note.split()) * truncate_note_tokens_at/ num_patient_note_tokens)

    # get truncated note
    result_list = patient_note.split()[:truncate_note_words_at]
    truncated_note = " ".join(result_list) + '...} ' # add this to show that the note continues

    return(truncated_note)

# The full prompt looks like this. But {input_text} needs to be truncated by the function above

prompt_template = """A virtual assistant answers questions from a user based on the provided text.
USER: Text: {input_text}
ASSISTANT: I’ve read this text.
USER: What describes {entity_name} in the text?
ASSISTANT: (model's predictions in JSON format)
"""


instructions = ''' } ASSISTANT: I’ve read this text.
USER: What describes {entity_name} in the text?
ASSISTANT: (model's predictions in JSON format)'''


In [111]:
# shorten_note(patient_note, instructions, max_length)



"Based on the patient's history and examination findings, I suspect that he may be experiencing a severe migraine attack. The patient's symptoms of headache, nausea, and vomiting are consistent with this diagnosis. Additionally, the patient's slow and deliberate movements and slurred speech suggest possible cerebellar dysfunction, which can occur in some cases of migraine.Based on these findings, I recommend the following plan:1. Admit the patient to the hospital for further evaluation and management.2. Order a CT scan of the head to rule out any intracranial causes of the patient's symptoms.3. Provide the patient with a triage of pain medications, including ibuprofen and sumatriptan, to help manage his headache.4. Monitor the patient's vital signs and fluid status closely, and provide appropriate fluids and electrolyte replacement as needed.5. Consider prescribing a preventive medication for the patient, such as a triptan or an ergotamine, to help reduce the frequency and severity of 

In [113]:
# What describes waiting?
treatment_plan = notes_data[0][list(notes_data[0].keys())[0]]['assessment_and_plan']
prompt = prompt_template.format_map({"input_text": shorten_note(treatment_plan, instructions, max_length), "entity_name": "waiting"})
print(prompt)
output = llm_ner(prompt)
output

A virtual assistant answers questions from a user based on the provided text.
USER: Text: Based on the patient's history and examination findings, I suspect that he may be experiencing a severe migraine attack. The patient's symptoms of headache, nausea, and vomiting are consistent with this diagnosis. Additionally, the patient's slow and deliberate movements and slurred speech suggest possible cerebellar dysfunction, which can occur in some cases of migraine.Based on these findings, I recommend the following plan:1. Admit the patient to the hospital for further evaluation and management.2. Order a CT scan of the head to rule out any intracranial causes of the patient's symptoms.3. Provide the patient with a triage of pain medications, including ibuprofen and sumatriptan, to help manage his headache.4. Monitor the patient's vital signs and fluid status closely, and provide appropriate fluids and electrolyte replacement as needed.5. Consider prescribing a preventive medication for the p

Llama.generate: prefix-match hit


[]


llama_print_timings:        load time =    5509.32 ms
llama_print_timings:      sample time =       0.18 ms /     2 runs   (    0.09 ms per token, 11235.96 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =     404.47 ms /     2 runs   (  202.23 ms per token,     4.94 tokens per second)
llama_print_timings:       total time =     407.72 ms


'[]'

In [114]:
# What describes treatment?
treatment_plan = notes_data[0][list(notes_data[0].keys())[0]]['assessment_and_plan']
prompt = prompt_template.format_map({"input_text": shorten_note(treatment_plan, instructions, max_length), "entity_name": "treatment"})
print(prompt)
output = llm_ner(prompt)
output

A virtual assistant answers questions from a user based on the provided text.
USER: Text: Based on the patient's history and examination findings, I suspect that he may be experiencing a severe migraine attack. The patient's symptoms of headache, nausea, and vomiting are consistent with this diagnosis. Additionally, the patient's slow and deliberate movements and slurred speech suggest possible cerebellar dysfunction, which can occur in some cases of migraine.Based on these findings, I recommend the following plan:1. Admit the patient to the hospital for further evaluation and management.2. Order a CT scan of the head to rule out any intracranial causes of the patient's symptoms.3. Provide the patient with a triage of pain medications, including ibuprofen and sumatriptan, to help manage his headache.4. Monitor the patient's vital signs and fluid status closely, and provide appropriate fluids and electrolyte replacement as needed.5. Consider prescribing a preventive medication for the p

Llama.generate: prefix-match hit


["pain medications", "ibuprofen", "sumatriptan", "fluids", "electrolyte replacement", "preventive medication", "triptan", "ergotamine"]


llama_print_timings:        load time =    5509.32 ms
llama_print_timings:      sample time =      14.61 ms /    47 runs   (    0.31 ms per token,  3217.86 tokens per second)
llama_print_timings: prompt eval time =    1014.00 ms /    20 tokens (   50.70 ms per token,    19.72 tokens per second)
llama_print_timings:        eval time =    3755.33 ms /    46 runs   (   81.64 ms per token,    12.25 tokens per second)
llama_print_timings:       total time =    5037.23 ms


'["pain medications", "ibuprofen", "sumatriptan", "fluids", "electrolyte replacement", "preventive medication", "triptan", "ergotamine"]'

### Using llama to identify what patient is waiting for

In [ ]:
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = Ollama(model="llama2", callback_manager=callback_manager)

In [67]:
template = """[INST]
<<SYS>>
You will be given a note or notes about a patient in hospital. 
From the notes, identify what the patient is currently waiting for.

Here is the information about the patient: 

{data}
[/INST]
"""

prompt = PromptTemplate.from_template(template)
# chain = prompt | llm

{'data' : notes_data[0]}

formatted_prompt = prompt.format_prompt(data = {'data' : notes_data[0]})
response = llm(formatted_prompt.to_string())

Based on the information provided in the notes, the patient is currently waiting for the results of a CT scan of the head to rule out any intracranial causes of their symptoms.

In [68]:
template = """[INST]
<<SYS>>
You will be given a note or notes about a patient in hospital. 
From the notes, identify what the patient is currently waiting for. There may be more than one thing they are waiting for.

Here is the information about the patient: 

{data}
[/INST]
"""

prompt = PromptTemplate.from_template(template)
# chain = prompt | llm

{'data' : notes_data[0]}

formatted_prompt = prompt.format_prompt(data = {'data' : notes_data[0]})
response = llm(formatted_prompt.to_string())

Based on the information provided in the notes, the patient is currently waiting for:

1. Admission to the hospital for further evaluation and management: The doctor has recommended admitting the patient to the hospital for a more detailed evaluation and treatment plan.
2. A CT scan of the head to rule out any intracranial causes of the patient's symptoms: The doctor has ordered a CT scan to help identify any underlying causes of the patient's headache, nausea, and vomiting.
3. A triage of pain medications to help manage the patient's headache: The doctor has provided the patient with a triage of pain medications, including ibuprofen and sumatriptan, to help manage his headache symptoms in the short term.
4. Closely monitoring the patient's vital signs and fluid status: The doctor is closely monitoring the patient's vital signs and fluid status to ensure that he stays hydrated and stable during his hospitalization.
5. Consideration of prescribing a preventive medication for the patient

In [66]:
template = """[INST]
<<SYS>>
You are a highly experienced emergency room nurse whose job is to minimise delays to patient care. 
A note has just been written after a doctor examined of the patient. 
From the note, identify what procedures, medications or other actions need to be taken immediate. 
List these actions succinctly in a to-do list for your colleagues. Do not justify why the actions are needed.
Limit your list only to things that would be done in hospital, and only items that the doctor is sure about.
Only use the information provided in the note. Do not give additional diagnoses. 

Here is the information about the patient: 

{data}
[/INST]
"""

prompt = PromptTemplate.from_template(template)
# chain = prompt | llm

{'data' : notes_data[0]}

formatted_prompt = prompt.format_prompt(data = {'data' : notes_data[0]})
response = llm(formatted_prompt.to_string())


Here is the list of actions that need to be taken immediate based on the information provided in the note:

1. Admit the patient to the hospital for further evaluation and management.
2. Order a CT scan of the head to rule out any intracranial causes of the patient's symptoms.
3. Provide the patient with a triage of pain medications, including ibuprofen and sumatriptan, to help manage his headache.
4. Monitor the patient's vital signs and fluid status closely, and provide appropriate fluids and electrolyte replacement as needed.

In [28]:
batch = []

for note_ in notes_data:
    batch.append({"data": note_})

In [29]:
outputs = chain.batch(batch)

Immediate actions needed:

1. Admit the patient to the hospital for further evaluation and management.
2. Order a CT scan of the head to rule out any intracranial causes of the patient's symptoms.
3. Provide the patient with a triage of pain medications, including ibuprofen and sumatriptan, to help manage his headache.
4. Monitor the patient's vital signs and fluid status closely, and provide appropriate fluids and electrolyte replacement as needed.Immediate actions required:

1. Admit the patient to the hospital for further observation and treatment.
2. Provide intravenous fluids to prevent dehydration.
3. Administer non-steroidal anti-inflammatory drugs (NSAIDs) for pain management.
4. Initiate a sumatriptan pain management plan.
5. Check the patient's blood work for any signs of electrolyte imbalances or other underlying conditions.Immediate actions required based on the information provided in the note:

1. Order urgent imaging studies (chest X-ray or CT scan) to confirm the presen

In [65]:
note_dict = {}

for i in range(0,len(batch)):
    key = list(notes_data[i].keys())[0]
    values = {'admission_note' : list(notes_data[i].values())[0], 
              'immediate actions' : outputs[i]}
    note_dict[key] = values


from utils.write_to_json import write_to_json
write_to_json(note_dict, notebook_name)




